# 2D and 3D Face Alignment Network
**2D FAN:**

![alt text](https://github.com/sony/nnabla-examples/raw/master/facial-keypoint-detection/face-alignment/results/example1.png)![alt text](https://github.com/sony/nnabla-examples/raw/master/facial-keypoint-detection/face-alignment/results/example2.png)

**3D FAN:**
![alt text](https://github.com/sony/nnabla-examples/raw/master/facial-keypoint-detection/face-alignment/results/example1_3d.png)![alt text](https://github.com/sony/nnabla-examples/raw/master/facial-keypoint-detection/face-alignment/results/example2_3d.png)

Here we'll show you a facial landmark detection example using [FAN (Face Alignment Network)](https://arxiv.org/pdf/1703.07332.pdf). 

# Preparation

First, we need to have [nnabla-examples repository](https://github.com/sony/nnabla-examples) and install nnabla. The following cell does both. Also, when you're running on Colab, make sure that your Runtime setting is set as GPU, which can be set up from the top menu (Runtime → change runtime type), and make sure to click **Connect** on the top right-hand side of the screen before you start.

In [ ]:
!pip install nnabla-ext-cuda100
!git clone https://github.com/sony/nnabla-examples.git
%run nnabla-examples/interactive-demos/colab_utils.py
%cd nnabla-examples/facial-keypoint-detection/face-alignment

Then, we need to download the pretrained weights and face detection model.

In [ ]:
# get 2D FAN pretrained weights.
!wget https://nnabla.org/pretrained-models/nnabla-examples/face-alignment/2DFAN4_NNabla_model.h5

# to run the 3D facial landmark detection, we need to have additional weights as well.
!wget https://nnabla.org/pretrained-models/nnabla-examples/face-alignment/3DFAN4_NNabla_model.h5
!wget https://nnabla.org/pretrained-models/nnabla-examples/face-alignment/Resnet_Depth_NNabla_model.h5

# get dlib's face detection model.
!wget http://dlib.net/files/mmod_human_face_detector.dat.bz2

# Upload an image

Run the below cell to upload an image to use FAN. Make sure to select **just 1 image** (if you upload multiple images, all the images but the last one will be ignored) and that image must contain at least one face.

In [ ]:
from google.colab import files

img = files.upload()

For convenience, rename the image file.

In [ ]:
import os
ext = os.path.splitext(list(img.keys())[-1])[-1]
os.rename(list(img.keys())[-1], "input_image{}".format(ext)) 

input_img = "input_image" + ext

# Run 2D FAN & Visualize the result

Now that we have the image to use, let's run 2D FAN and see the result. The following cell executes FAN network and generates the output image named "output_image.png".

In [ ]:
!python model_inference.py --model 2DFAN4_NNabla_model.h5 --test-image $input_img --output output_image.png

from IPython.display import Image,display
display(Image('output_image.png'))

# Run 3D FAN & Visualize the result

Next, let's see what happens if we use 3D landmark detection. The following cell executes 3D FAN and generate the output image named "output_image_3D.png".

In [ ]:
!python model_inference.py --landmarks-type-3D --model 3DFAN4_NNabla_model.h5 --resnet-depth-model Resnet_Depth_NNabla_model.h5 --test-image $input_img --output output_image_3D.png

display(Image('output_image_3D.png'))

# Visualize the 3D FAN result with 3D plot

The image above shows the keypoints, but since it is rendered on 2D image it's hard to tell the difference from the 2D result. So next we use mplot3d to see how the detected keypoints are represented in the 3D space. Note that the script below is a partially modified version of model_inference.py [as of ver 1.9.0](https://github.com/sony/nnabla-examples/blob/release/v1.9.0-branch/facial-keypoint-detection/face-alignment/model_inference.py), so there might be some difference in the later version.


In [ ]:
#@title First let's start by importing dependencies. (double-click to see the codes)
import cv2
import dlib
import nnabla as nn
import nnabla.functions as F
from skimage import io, color
from model import fan, resnet_depth
from external_utils import *
import numpy as np

This is the main part of the keypoint detection. Using the uploaded images above, we first detect the face region with dlib, then crop that area to extract the facial image, and finally run the FAN to get the predicted keypoints.

In [ ]:
#@title Execute Face Detection and FAN. (double-click to see the codes)
from nnabla.ext_utils import get_extension_context
ctx = get_extension_context("cudnn")
nn.set_default_context(ctx)
nn.set_auto_forward(True)

image = io.imread(input_img)
if image.ndim == 2:
    image = color.gray2rgb(image)
elif image.ndim == 4:
    image = image[..., :3]

face_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")
detected_faces = face_detector(cv2.cvtColor(image[..., ::-1].copy(), cv2.COLOR_BGR2GRAY))
detected_faces = [[d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom()] for d in detected_faces]

if len(detected_faces) == 0:
    print("Warning: No faces were detected.")
    sys.exit()

# Load FAN weights
with nn.parameter_scope("FAN"):
    print("Loading FAN weights...")
    nn.load_parameters("3DFAN4_NNabla_model.h5")

# Load ResNetDepth weights
with nn.parameter_scope("ResNetDepth"):
    print("Loading ResNetDepth weights...")
    nn.load_parameters("Resnet_Depth_NNabla_model.h5")

landmarks = []
for i, d in enumerate(detected_faces):
    center = [d[2] - (d[2] - d[0]) / 2.0, d[3] - (d[3] - d[1]) / 2.0]
    center[1] = center[1] - (d[3] - d[1]) * 0.12
    scale = (d[2] - d[0] + d[3] - d[1]) / 195
    inp = crop(image, center, scale)
    inp = nn.Variable.from_numpy_array(inp.transpose((2, 0, 1)))
    inp = F.reshape(F.mul_scalar(inp, 1 / 255.0), (1,) + inp.shape)
    with nn.parameter_scope("FAN"):
        out = fan(inp, 4)[-1]
    pts, pts_img = get_preds_fromhm(out, center, scale)
    pts, pts_img = F.reshape(pts, (68, 2)) * \
        4, F.reshape(pts_img, (68, 2))

    heatmaps = np.zeros((68, 256, 256), dtype=np.float32)
    for i in range(68):
        if pts.d[i, 0] > 0:
            heatmaps[i] = draw_gaussian(
                heatmaps[i], pts.d[i], 2)
    heatmaps = nn.Variable.from_numpy_array(heatmaps)
    heatmaps = F.reshape(heatmaps, (1,) + heatmaps.shape)
    with nn.parameter_scope("ResNetDepth"):
        depth_pred = F.reshape(resnet_depth(
            F.concatenate(inp, heatmaps, axis=1)), (68, 1))
    pts_img = F.concatenate(
        pts_img, depth_pred * (1.0 / (256.0 / (200.0 * scale))), axis=1)


We now prepare for the visualization with mplot3d. The code below is from [the original author's repository](https://github.com/1adrianb/face-alignment/blob/master/examples/detect_landmarks_in_image.py).

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import collections

In [ ]:
pred_type = collections.namedtuple('prediction_type', ['slice', 'color'])
pred_types = {'face': pred_type(slice(0, 17), (0.682, 0.780, 0.909, 0.5)),
              'eyebrow1': pred_type(slice(17, 22), (1.0, 0.498, 0.055, 0.4)),
              'eyebrow2': pred_type(slice(22, 27), (1.0, 0.498, 0.055, 0.4)),
              'nose': pred_type(slice(27, 31), (0.345, 0.239, 0.443, 0.4)),
              'nostril': pred_type(slice(31, 36), (0.345, 0.239, 0.443, 0.4)),
              'eye1': pred_type(slice(36, 42), (0.596, 0.875, 0.541, 0.3)),
              'eye2': pred_type(slice(42, 48), (0.596, 0.875, 0.541, 0.3)),
              'lips': pred_type(slice(48, 60), (0.596, 0.875, 0.541, 0.3)),
              'teeth': pred_type(slice(60, 68), (0.596, 0.875, 0.541, 0.4))
              }

In [ ]:
#@title Visualize the result. (double-click to see the codes)
# uncomment the line below if you run this in jupyter notebook.
#matplotlib notebook

fig = plt.figure(figsize=plt.figaspect(.5))
ax = fig.add_subplot(1, 2, 2, projection='3d')
surf = ax.scatter(pts_img.d[:, 0] * 1.2,
                  pts_img.d[:, 1],
                  pts_img.d[:, 2],
                  c='cyan',
                  alpha=1.0,
                  edgecolor='b')

for pred_type in pred_types.values():
    ax.plot3D(pts_img.d[pred_type.slice, 0] * 1.2,
              pts_img.d[pred_type.slice, 1],
              pts_img.d[pred_type.slice, 2], color='blue')

ax.view_init(elev=100., azim=90.)
ax.set_xlim(ax.get_xlim()[::-1])
plt.show()

# For Smartphone Users
If you're using a smartphone with a camera, you can take a photo and use that image for this demo.
Just execute the following cell and tap `Capture` button. 

If your device has multiple cameras (such as front and back) you need to select which one to use by tapping the corresponding button (which should appear near the 'Capture' button).

**Note this is an experimental support and may not work in some devices.**


In [ ]:
try:
    filename = take_photo(cam_width=256, cam_height=256)
    print('Saved to {}'.format(filename))
    # Show the image which was just taken.
    display(Image(filename))
except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

If the photo is OK, let's run (2D) FAN. If you want to use another photo, just re-run the previous cell again.

The following cell will execute the same as done above on your photo.

In [ ]:
!python model_inference.py --model 2DFAN4_NNabla_model.h5 --test-image $input_img --output output_image.png
from IPython.display import Image,display
display(Image('output_image.png'))